In [66]:
# 计算每个点之间的距离
from collections import defaultdict

class Solution:
    def shortestSuperstring(self, A):
        def calc(word1, word2): # 计算两个单词之间的 distance
            n1, n2 = len(word1), len(word2)
            right = 0
            for i in range(n1):
                if word1[i] == word2[right]:
                    ans = n2
                    idx = i
                    while word1[idx] == word2[right] and idx < n1 and right < n2:
                        idx += 1
                        right += 1
                        ans -= 1
                        if idx == n1:
                            return ans
            return n2
        
        def get_min_path():
            def dfs(arr, path):
                if not arr:
                    res.append(path[:])
                    return 
                for i in range(len(arr)):
                    path.append(arr[i])
                    dfs(arr[:i] + arr[i+1:], path)
                    path.pop()
                    
            res = []
            nums = [i for i in range(n)]
            dfs(nums, [])
            outs = []
            min_dis = float('inf')
            for per in res:
                cur_dis = 0
                for k in range(n-1):
                    cur_dis += dis[per[k]][per[k+1]]
                if cur_dis < min_dis:
                    min_dis = cur_dis
                    outs = per
            print(per)
            return outs
        
        n = len(A)
        dis = defaultdict(dict)
        for i in range(n): # 计算每个单词之间的距离
            for j in range(i+1, n):
                dis[i][j] = calc(A[i], A[j])
                dis[j][i] = calc(A[j], A[i]) #计算两个单词之间的距离
        print(dis)
        res = ''
        path = get_min_path()
        for i in path:
            res += A[i]
        return res

In [204]:
class Solution:
    def shortestSuperstring(self, A):
        def calc_dis(w1, w2): # 计算w1 和 w2 之间的距离
            n1, n2 = len(w1), len(w2)
            for k in range(min(n1, n2), -1, -1):
                if w1[-k:] == w2[:k]:
                    return n2 - k
            return n2
        
        def combine(cur_res, word): # 重构为 答案
            n1, n2 = len(cur_res), len(word)
            for k in range(min(n1, n2), -1, -1):
                if cur_res[-k:] == word[:k]:
                    return cur_res + word[k:]
            return cur_res + word
        
        N = len(A) 
        M = 1 << N
        dp = [[float('inf')] * N for _ in range(M)] # dp[0111][last] 
        parent = [[-1] * N for _ in range(M)] 
        
        # 计算每个城市之间的距离
        dis = [[0] * N for _ in range(N)] 
        for i in range(N):
            for j in range(N):
                if i != j:
                    dis[i][j] = calc_dis(A[i], A[j])
        
        # 如果只行走一个城市，那么dis == len(A[i])
        for i in range(N):
            dp[1 << i][i] = len(A[i])
        
        for cur_state in range(M): # 遍历各种状态 0000 -> 1111: 每个城市都没走->全部都走过
            for cur in range(N): # state: 0111, last: 可能是 B、C、D中的一个
                if (cur_state & (1 << cur)) == 0: # 当前城市还没有走
                    continue
                pre_state = cur_state - (1 << cur) # 当前状态 0111，last = B, 则 有可能是 0011[D], 0011[C] 转移到的B
                
                if pre_state == 0: # 有可能只走了一个城市
                    continue
                for prev in range(N):
                    if (pre_state & (1 << prev)) == 0: # 当前城市还没有走
                        continue
                    if dp[pre_state][prev] + dis[prev][cur] < dp[cur_state][cur]:
                        dp[cur_state][cur] = dp[pre_state][prev] + dis[prev][cur]
                        parent[cur_state][cur] = prev
        
        start = None # 起点
        min_dis = float('inf')
        for i in range(N):
            if dp[-1][i] < min_dis:
                min_dis = dp[-1][i]
                start = i
                
        mask = M - 1 # 根据起点和 mask 回溯路径
        path = [start]
        while parent[mask][start] != -1:
            nxt = parent[mask][start]
            path.append(nxt)
            print(bin(mask)[2:], start, 'before')
            mask = mask ^ (1 << start)
            print(bin(mask)[2:], start, 'after')
            start = nxt
        
        path = path[::-1] # 整合字符串
        res = A[path[0]]
        for i in range(1, len(path)):
            res = combine(res, A[path[i]])
        return res

### 12-10 第二遍

In [115]:
class Solution:
    def shortestSuperstring(self, A):
        def calc_dis(w1, w2): # 计算 w1 和 w2 之间的距离
            n1, n2 = len(w1), len(w2)
            for k in range(min(n1, n2), -1, -1):
                if w1[-k:] == w2[:k]:
                    return n2 - k
            return n2

        def combine(cur_res, word): # 拼接字符串
            n1, n2 = len(cur_res), len(word)
            for k in range(min(n1, n2), -1, -1):
                if cur_res[-k:] == word[:k]:
                    return cur_res + word[k:]
            return cur_res + word

        n = len(A)
        m = 1 << n # 一共有多少种可能性
        # dp[0011][cur_city] 经过 0011 -> (AB) CD 去过CD两地，并且最后一个去的地方是 cur_city
        dp = [[float('inf')] * n for _ in  range(m)]
        par = [[-1] * n for _ in  range(m)] # 记录达到此处时，上一处在哪里
        for i in range(n): # 只去过一个城市的时候，cost == len(A[i])
            dp[1 << i][i] = len(A[i])
        
        dis = [[0] * n for _ in range(n)]
        for i in range(n):
            for j in range(n):
                if i != j:
                    dis[i][j] = calc_dis(A[i], A[j])
        
        for state in range(m): # 当前的状态
            for cur in range(n):
                if (state & (1 << cur)) == 0:
                    continue
                prev_state = state - (1 << cur)
                if prev_state == 0: # 如果只去过一个城市，那么prev_state == 0, 是否只是去过一个城市
                    continue
                for prev in range(n):
                    if (prev_state & (1 << prev)) == 0:
                        continue
                    if dp[prev_state][prev] + dis[prev][cur] < dp[state][cur]:
                        dp[state][cur] = dp[prev_state][prev] + dis[prev][cur]
                        par[state][cur] = prev
        
        min_dis = float('inf')
        for i in range(n):
            if dp[-1][i] < min_dis:
                last_city = i # 最后去过的city 在哪里
                min_dis = dp[-1][i] # 最短路径为多少
        
        # 回溯最短路径
        path = [last_city]
        mask = m - 1
        while par[mask][last_city] != -1:
            prev_city = par[mask][last_city]
            path.append(prev_city)
            mask = mask ^ (1 << last_city)
            last_city = prev_city
        
        path = path[::-1]
        res = A[path[0]]
        for i in range(1, len(path)):
            res = combine(res, A[path[i]])
        return res

In [116]:
solution = Solution()
solution.shortestSuperstring(["sssv","svq","dskss","sksss"])

'dsksssvq'

In [98]:
a = 'abcd'
a[-1:]

'd'

In [168]:
len("dsksssvq")

8

In [131]:
bin(8)

'0b1000'